In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import glob
import os
import re
import csv

In [2]:
df = pd.read_csv('csv.csv', engine='python')
print(df)

            date     value
0     2010-07-26  10525.43
1     2010-07-27  10537.69
2     2010-07-28  10497.88
3     2010-07-29  10467.16
4     2010-07-30  10465.94
...          ...       ...
3287  2023-08-07       NaN
3288  2023-08-08       NaN
3289  2023-08-09       NaN
3290  2023-08-10       NaN
3291  2023-08-11       NaN

[3292 rows x 2 columns]


In [3]:
from datetime import date


df['date'] = pd.to_datetime(df['date'])

start_date = pd.Timestamp(2016, 12, 30)
end_date = pd.Timestamp(2021, 12, 31)


In [4]:
df = df.loc[df['date'] >= start_date]
df = df.loc[df['date'] <= end_date]
print(df)

           date     value
1623 2016-12-30  19762.60
1624 2017-01-03  19881.76
1625 2017-01-04  19942.16
1626 2017-01-05  19899.29
1627 2017-01-06  19963.80
...         ...       ...
2876 2021-12-27  36302.38
2877 2021-12-28  36398.21
2878 2021-12-29  36488.63
2879 2021-12-30  36398.08
2880 2021-12-31  36338.30

[1258 rows x 2 columns]


In [5]:
for i in df['date']:
    print(i)

2016-12-30 00:00:00
2017-01-03 00:00:00
2017-01-04 00:00:00
2017-01-05 00:00:00
2017-01-06 00:00:00
2017-01-09 00:00:00
2017-01-10 00:00:00
2017-01-11 00:00:00
2017-01-12 00:00:00
2017-01-13 00:00:00
2017-01-17 00:00:00
2017-01-18 00:00:00
2017-01-19 00:00:00
2017-01-20 00:00:00
2017-01-23 00:00:00
2017-01-24 00:00:00
2017-01-25 00:00:00
2017-01-26 00:00:00
2017-01-27 00:00:00
2017-01-30 00:00:00
2017-01-31 00:00:00
2017-02-01 00:00:00
2017-02-02 00:00:00
2017-02-03 00:00:00
2017-02-06 00:00:00
2017-02-07 00:00:00
2017-02-08 00:00:00
2017-02-09 00:00:00
2017-02-10 00:00:00
2017-02-13 00:00:00
2017-02-14 00:00:00
2017-02-15 00:00:00
2017-02-16 00:00:00
2017-02-17 00:00:00
2017-02-21 00:00:00
2017-02-22 00:00:00
2017-02-23 00:00:00
2017-02-24 00:00:00
2017-02-27 00:00:00
2017-02-28 00:00:00
2017-03-01 00:00:00
2017-03-02 00:00:00
2017-03-03 00:00:00
2017-03-06 00:00:00
2017-03-07 00:00:00
2017-03-08 00:00:00
2017-03-09 00:00:00
2017-03-10 00:00:00
2017-03-13 00:00:00
2017-03-14 00:00:00


In [6]:
df['day_of_week'] = df['date'].dt.day_name()

In [7]:
print(df)

           date     value day_of_week
1623 2016-12-30  19762.60      Friday
1624 2017-01-03  19881.76     Tuesday
1625 2017-01-04  19942.16   Wednesday
1626 2017-01-05  19899.29    Thursday
1627 2017-01-06  19963.80      Friday
...         ...       ...         ...
2876 2021-12-27  36302.38      Monday
2877 2021-12-28  36398.21     Tuesday
2878 2021-12-29  36488.63   Wednesday
2879 2021-12-30  36398.08    Thursday
2880 2021-12-31  36338.30      Friday

[1258 rows x 3 columns]


In [8]:
day_filter = df['day_of_week'] == 'Friday'
fridays = df.loc[day_filter]
print(fridays)

           date     value day_of_week
1623 2016-12-30  19762.60      Friday
1627 2017-01-06  19963.80      Friday
1632 2017-01-13  19885.73      Friday
1636 2017-01-20  19827.25      Friday
1641 2017-01-27  20093.78      Friday
...         ...       ...         ...
2856 2021-11-26  34899.34      Friday
2861 2021-12-03  34580.08      Friday
2866 2021-12-10  35970.99      Friday
2871 2021-12-17  35365.44      Friday
2880 2021-12-31  36338.30      Friday

[253 rows x 3 columns]


In [9]:
fridays = fridays.reset_index(drop=True)
#fridays['change'] = 0
print(fridays)

          date     value day_of_week
0   2016-12-30  19762.60      Friday
1   2017-01-06  19963.80      Friday
2   2017-01-13  19885.73      Friday
3   2017-01-20  19827.25      Friday
4   2017-01-27  20093.78      Friday
..         ...       ...         ...
248 2021-11-26  34899.34      Friday
249 2021-12-03  34580.08      Friday
250 2021-12-10  35970.99      Friday
251 2021-12-17  35365.44      Friday
252 2021-12-31  36338.30      Friday

[253 rows x 3 columns]


In [10]:
change = [0]
percentChange = [0]

for index, row in fridays.iterrows():
    print(index)
    if row['date'] != pd.to_datetime('2016-12-30'):
        #print(index)
        tempRow = fridays.iloc[[index-1]]
        #print(tempRow)
        #print(type(tempRow))
        #print(tempRow.columns)
        #prevVal = tempRow.get('value')
        prevVal = tempRow[' value']
        #print(prevVal)
        weekChange = pd.to_numeric(row[' value']-prevVal)
        print(weekChange)
        print(weekChange.values[0])
        weekChange = weekChange.values[0]
        print(type(weekChange))
        change.append(weekChange)
        
        valPercent = row[' value']
        #print(valPercent)
        changePercent = weekChange/valPercent
        print(changePercent)
        percentChange.append(changePercent)
        
        print("KLJSDHFDJSFNKJDSHGFNG")

0
1
0    201.2
Name:  value, dtype: float64
201.20000000000073
<class 'numpy.float64'>
0.0100782416173274
KLJSDHFDJSFNKJDSHGFNG
2
1   -78.07
Name:  value, dtype: float64
-78.06999999999971
<class 'numpy.float64'>
-0.003925930805658113
KLJSDHFDJSFNKJDSHGFNG
3
2   -58.48
Name:  value, dtype: float64
-58.47999999999956
<class 'numpy.float64'>
-0.002949476099812105
KLJSDHFDJSFNKJDSHGFNG
4
3    266.53
Name:  value, dtype: float64
266.52999999999884
<class 'numpy.float64'>
0.013264303680044215
KLJSDHFDJSFNKJDSHGFNG
5
4   -22.32
Name:  value, dtype: float64
-22.31999999999971
<class 'numpy.float64'>
-0.0011120267284990583
KLJSDHFDJSFNKJDSHGFNG
6
5    197.91
Name:  value, dtype: float64
197.90999999999985
<class 'numpy.float64'>
0.009763993651504703
KLJSDHFDJSFNKJDSHGFNG
7
6    354.68
Name:  value, dtype: float64
354.6800000000003
<class 'numpy.float64'>
0.01719739818318906
KLJSDHFDJSFNKJDSHGFNG
8
7    197.71
Name:  value, dtype: float64
197.70999999999913
<class 'numpy.float64'>
0.00949535485

85   -48.28
Name:  value, dtype: float64
-48.279999999998836
<class 'numpy.float64'>
-0.0018629029955387114
KLJSDHFDJSFNKJDSHGFNG
87
86    238.13
Name:  value, dtype: float64
238.12999999999738
<class 'numpy.float64'>
0.009104683790695787
KLJSDHFDJSFNKJDSHGFNG
88
87    588.83
Name:  value, dtype: float64
588.8300000000017
<class 'numpy.float64'>
0.022017686540654804
KLJSDHFDJSFNKJDSHGFNG
89
88   -285.19
Name:  value, dtype: float64
-285.1899999999987
<class 'numpy.float64'>
-0.010778844151421564
KLJSDHFDJSFNKJDSHGFNG
90
89   -11.26
Name:  value, dtype: float64
-11.260000000002037
<class 'numpy.float64'>
-0.00042575636980313636
KLJSDHFDJSFNKJDSHGFNG
91
90   -1107.06
Name:  value, dtype: float64
-1107.0599999999977
<class 'numpy.float64'>
-0.04368825717768624
KLJSDHFDJSFNKJDSHGFNG
92
91    104.35
Name:  value, dtype: float64
104.34999999999854
<class 'numpy.float64'>
0.004101108537301362
KLJSDHFDJSFNKJDSHGFNG
93
92   -756.03
Name:  value, dtype: float64
-756.0299999999988
<class 'numpy.f

170
169   -51.58
Name:  value, dtype: float64
-51.580000000001746
<class 'numpy.float64'>
-0.0021741980273727126
KLJSDHFDJSFNKJDSHGFNG
171
170    607.63
Name:  value, dtype: float64
607.630000000001
<class 'numpy.float64'>
0.024973162163006407
KLJSDHFDJSFNKJDSHGFNG
172
171   -645.9
Name:  value, dtype: float64
-645.9000000000015
<class 'numpy.float64'>
-0.02726994074835918
KLJSDHFDJSFNKJDSHGFNG
173
172    779.74
Name:  value, dtype: float64
779.7400000000016
<class 'numpy.float64'>
0.03187144494456613
KLJSDHFDJSFNKJDSHGFNG
174
173    917.95
Name:  value, dtype: float64
917.9500000000007
<class 'numpy.float64'>
0.03616381129026351
KLJSDHFDJSFNKJDSHGFNG
175
174    1727.87
Name:  value, dtype: float64
1727.869999999999
<class 'numpy.float64'>
0.06373321805408727
KLJSDHFDJSFNKJDSHGFNG
176
175   -1505.44
Name:  value, dtype: float64
-1505.4399999999987
<class 'numpy.float64'>
-0.058793526713359634
KLJSDHFDJSFNKJDSHGFNG
177
176    265.92
Name:  value, dtype: float64
265.91999999999825
<class

In [11]:
fridays['change'] = change
fridays['percent_change'] = percentChange
print(fridays)

          date     value day_of_week   change  percent_change
0   2016-12-30  19762.60      Friday     0.00        0.000000
1   2017-01-06  19963.80      Friday   201.20        0.010078
2   2017-01-13  19885.73      Friday   -78.07       -0.003926
3   2017-01-20  19827.25      Friday   -58.48       -0.002949
4   2017-01-27  20093.78      Friday   266.53        0.013264
..         ...       ...         ...      ...             ...
248 2021-11-26  34899.34      Friday  -702.64       -0.020133
249 2021-12-03  34580.08      Friday  -319.26       -0.009232
250 2021-12-10  35970.99      Friday  1390.91        0.038668
251 2021-12-17  35365.44      Friday  -605.55       -0.017123
252 2021-12-31  36338.30      Friday   972.86        0.026772

[253 rows x 5 columns]


In [12]:
fridays.to_csv('DJIA_Friday_Changes.csv', index=False)